In [58]:
import cv2
import numpy as np
import scipy.ndimage
from scipy.ndimage.morphology import distance_transform_edt
from google.colab.patches import cv2_imshow

def MainFunction(JPEGimage, image, number=''):
    img_ = cv2.imread(JPEGimage)
    img1 = cv2.cvtColor(img_,cv2.COLOR_BGR2GRAY)

    img = cv2.imread(image)
    img2 = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    sift = cv2.xfeatures2d.SIFT_create()
    # find the key points and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    img1_result = cv2.drawKeypoints(img_,kp1,None, cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv2_imshow(img1_result)

    match = cv2.BFMatcher(cv2.NORM_L2, True)
    matches = match.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)
    res = cv2.drawMatches(img_,kp1,img,kp2,matches[:10],None,flags=2)
    cv2_imshow(res)
            
    pts1 = np.array([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1, 2)
    pts2 = np.array([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1, 2)
    M, _ = cv2.findHomography(pts2, pts1, cv2.RANSAC) 
    
    warped_image = cv2.warpPerspective(img ,M,(img_.shape[1] + img.shape[1], img_.shape[0]))
    warped_image = warped_image/2
    warped_image[0:img_.shape[0], 0: img_.shape[1]] += img_/2
    cv2.imwrite("not_blended.png", warped_image)
    warped_image = cv2.warpPerspective(img, M, (img_.shape[1] + img.shape[1], img_.shape[0]))
    warped_image[0:img_.shape[0], 0:img_.shape[1]] = img_
    cv2.imwrite("blended.png", warped_image)
  
    warped_image[0:img.shape[0], 0:img.shape[1]] = img_
    warped_image = warped_image
    warped_image = cv2.cvtColor(warped_image,cv2.COLOR_BGR2RGBA) 
  
    cv2_imshow(warped_image)

    warped_image2 = cv2.warpPerspective(img, M, (img_.shape[1] + img.shape[1], img_.shape[0]))
    warped_image1 = np.zeros(warped_image2.shape)
    warped_image1[0:img_.shape[0], 0:img_.shape[1]] = img_
    distance1 = distance_transform_edt(warped_image1)
    distance2 = distance_transform_edt(warped_image2)

    w1 = distance1 / np.maximum((distance1+distance2),1)
    w2 = distance2 / np.maximum((distance1+distance2),1)
    warped_image = warped_image1*w1 + warped_image2*w2
    warped_image = warped_image[0:warped_image.shape[0], 0: warped_image.shape[1]//5]
    cv2_imshow(warped_image)
    cv2.imwrite("self_final.png", warped_image)
    print(warped_image.shape)

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

img1 = cv2.imread('/content/img1.jpg')
img2 = cv2.imread('/content/img2.jpg')
img3 = cv2.imread('/content/img3.jpg')
img4 = cv2.imread('/content/img4.jpg')

imgList = [img1, img2, img3, img4] 

for i in range(4):
  resize = image_resize(imgList[i], imgList[i].shape[0]//3, imgList[i].shape[1]//3)
  cv2.imwrite("resize_img%d.jpg"%(i), resize)  
    


MainFunction('/content/self4.png', '/content/self5.png')


Output hidden; open in https://colab.research.google.com to view.

Reasonings:

I took my photo outdoor since there are more patterns and dynamic points that could overlap with other photos. Moreover, I thought it would be a lot harder to validate my model with indoor photos that contains plane walls with small characteristics since the model would work perfectly anytime, which would be difficult to improve upon our model. 

The first flaw that I encountered with the first attempt of stitching my own photos was that the model would not generate panorama photos that had different angles. For instance, when I was taking a photo of our campus just by altering my wrist angles, the model that I implemented has produced a photo that was extremely inaccurate. However, like the model worked fine when I was moving sideways with still hand positions (like the panorama photo that I generated as my final_self.png. 

Another flaw that I encountered was that the panorama photos generated by the model had some issues towards the right side of the image. It seems like the photo has stretched out to the right. I also believe that this was caused by different angles between the photos, and possibly from the floor level being different. 

Lastly, if we look at final_self.png photo, it shows that the trees and plants in the middle part is blurred out, and they are not in the right position compared to the original photos. I believe this was also due to the difference in the elevation of the floor in img4 and img3. Therefore, the model has possibly taken the average homographic transformation within these two planes.

In order to make my model more robust, there would need to be a function that aligns the images accordingly to its angles. We could possibly use convNets to align our images by performing corners and edge dectections to align the two images. Moreover, we could use spatial transformer Networks to train and align two images by learning the transformation parameters taht map one image onto the other. 

